In [ ]:
from __future__ import division
from scipy.optimize import fsolve, root
from scipy import integrate
from sympy import Symbol, exp, log
from numpy import linalg as la
import matplotlib.pyplot as plt
import numpy as np
import time
import scipy.stats
import numpy.random as nrd
import multiprocessing

SampleNumber = 1
class HDFPCA_model():
    def __init__(self):
        self.h = 50
        self.w = 50
        self.NXI = 100 # self.NXI = 3  eigenscore的个数
        self.L = 10   # self.L = 3 拆分数量
        self.V = self.h * self.w
        self.pg = int(self.V / self.L) 
        
    def gen_observers(self,N=SampleNumber): # Num = SampleNumber
        eigenimage = np.zeros(shape=[self.NXI, self.h, self.w])
        part_setting = int(np.sqrt(self.h*self.w/self.NXI))
        for _1 in range(int(np.sqrt(self.NXI))):
            for _2 in range(int(np.sqrt(self.NXI))):
                eigenimage[int(np.sqrt(self.NXI))*_1+_2 , _1*part_setting:(_1+1)*part_setting, _2*part_setting:(_2+1)*part_setting] = 1 
        eigenimage = eigenimage / np.sqrt(np.mean(eigenimage**2, (1,2)))[:, np.newaxis, np.newaxis]
        sigma = np.zeros([self.NXI])
        eigenscore = np.zeros([N, self.NXI])
        # SVD
        for d in range(self.NXI):
            sigma[d] = np.power(0.95,d-1)#2-d*(2/self.NXI)
            eigenscore[:, d] = nrd.normal(0, np.sqrt(sigma[d]), size=N)
        observers = np.sum(eigenscore[:, :, np.newaxis, np.newaxis] * eigenimage, 1)  # N * h * w
        return eigenscore,observers

    def get_X_bar(self,X,N=SampleNumber): # x = observers
        X_bar = X.reshape((N, -1))/self.h   # transfer x to vector ( N * grid)
        X_bar = X_bar - np.mean(X_bar, 0)[np.newaxis]   # centered x
        return X_bar    
    
    def get_est_U(self,X,N=SampleNumber,return_eigenscore=False):
        # SVD
        X_bar = X.reshape((N, -1))/self.h   # transfer x to vector ( N * grid)
        X_bar = X_bar - np.mean(X_bar, 0)[np.newaxis]   # centered x
        X_bar_square = np.zeros(shape=[N, N])
        for l in range(self.L):
            X_bar_part = X_bar[:, l*self.pg:(l+1)*self.pg]
            X_bar_square += np.matmul(X_bar_part, X_bar_part.transpose())

        v, s_2, vt = la.svd(X_bar_square)  # u: n*n, s: n (eigenvalue)
        est_eimg = np.zeros([N, self.L, self.pg])
        for l in range(self.L):
            X_bar_part = X_bar[:, l*self.pg:(l+1)*self.pg]
            est_eimg[:, l] = np.matmul(X_bar_part.T, np.matmul(v, np.sqrt(np.diag(1/s_2)))).T # U(m)=X(m)VS_-1
        U = est_eimg.reshape((N, -1))
        if return_eigenscore==False:
            return U.T
        else:
            return np.matmul(U,X_bar.T)

